In [2]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy import linalg
import matplotlib.pyplot as plt
import random
from IPython.display import display, clear_output
from jupyterthemes import jtplot
import time
import scipy as scipy
import csv

jtplot.style(theme = 'gruvboxd')

In [19]:
# # simulation inputs 
# total_time = 100
# gr_sz = 0.1
# MAT = -5
# T_AMP = 7
# SLHL_C14 = 15.1
# SLHL_Be10 = 5.01
# SLHL_He3 = 114
# scaling_factor = 1
# save_output = True

# # erosion inputs
# scenarios = 3

In [4]:
# # CN production inputs
# He_depth = max_depth[0]
# rock_rho = 2.7
# CN_lambda = 160
# mu = rock_rho / CN_lambda
# dt = 1

# # He diffusion inputs
# rho = 2.65 # quartz density
# nx = 100
# r = gr_sz
# dx = (r / nx)
# x = np.arange(dx/2, r, dx)
# x_up = x + (dx/2)
# x_low = x - (dx/2)
# x_vol = [(4/3 * np.pi * x_up[i]**3) - (4/3 * np.pi * x_low[i]**3) for i in range(len(x))]
# shell_mass = [(x_vol[i] * rho) for i in range(len(x_vol))]

# # Bedrock thermal properties
# alpha = 1.5e-6

In [5]:
def rockfalls_import():
    read_in = open('D:/STEIN_paper/model_outputs/stochastic_erosion/erosion_sets/import_dates.csv')
    myreader = csv.reader(read_in)
    rockfall_mat = np.empty((total_time, scenarios))
    
    for row in myreader:
        mat_gen_dates = row[:]
    
    rockfall_mat = np.genfromtxt('D:/STEIN_paper/model_outputs/stochastic_erosion/erosion_sets/rockfall_matrix_' + str(stoch_base_ER) + '_' + str(mat_gen_dates[-1]) + '.csv',
                                  delimiter = ',')
    ERs = np.genfromtxt('D:/STEIN_paper/model_outputs/stochastic_erosion/erosion_sets/ER_list_' + str(stoch_base_ER) + '_' + str(mat_gen_dates[-1]) + '.csv',
                                  delimiter = ',')
        
    return rockfall_mat, ERs

In [2]:
def He3_temps(MAT, T_AMP, dat_lin):
    dat_lin = dat_lin
        
    T0 = Temp_fun(MAT, T_AMP, dat_lin, (1/thermnt))
    EDT_import = EDT_calculation(MAT, T_AMP, T0)
    EDT_trnsfrm = interp1d(x = EDT_import[5], y = EDT_import[3], 
                        kind = 'linear',
                        fill_value = "extrapolate")
    EDT_z = EDT_trnsfrm(x = dat_lin)
    
    return EDT_z

In [7]:
def He3_fxn(grain_conc, EDT_z, rockfall_mat, dat_lin):
    
    fall_thickness = rockfall_mat[0]    
    He3P0 = SLHL_He3 * np.exp(-mu * dat_lin)
    grain_conc = [MT_He_3(He3P0[i], grain_conc[i], x, 1, dx, EDT_z[i], nx) for i in range(len(dat_lin))] 

    depths = dat_lin[dat_lin > fall_thickness]
    
    first_fxn = interp2d(x = range(0,nx), 
                         y = dat_lin, 
                         z = np.stack(grain_conc), 
                         kind = 'linear')    
    
    first_data = first_fxn(x = range(0,nx), y = dat_lin)
    
    second_fxn = interp2d(x = range(0,nx), 
                          y = (dat_lin - fall_thickness), 
                          z = np.stack(first_data), 
                          kind = 'linear') 
    
    grain_conc = second_fxn(x = range(0,nx), y = dat_lin)
    
    return grain_conc

In [4]:
def He3_loop_fxn(total_time):
    dat_lin = np.arange(0,max_depth[0] + 1,1)
    
    %run C:\Users\dennis\STEIN_codes\helium\He_3_after_Tremblay.ipynb
    %run C:\Users\dennis\STEIN_codes\temperature\new_crank_nicolson_fixed.ipynb
    
    all_rockfalls, ERs = rockfalls_import()
    
    EDT_z = He3_temps(MAT, T_AMP, dat_lin)
    He_surf_mat = np.empty((total_time,scenarios))
    
    for i in range(scenarios):
        conc3He = [[0]*nx] * len(dat_lin)
        He_surf = np.empty((total_time,))
        
        
        year_falls = all_rockfalls[:,i]
        rockfall_plot = year_falls
        ER = ERs[i]
    
        for j in range(total_time):
            conc3He = He3_fxn(conc3He, EDT_z, year_falls, dat_lin)
            He_surf[j] = np.average([np.average(conc3He[i], weights = shell_mass) for i in range(6)])
        
        He_surf_mat[:,i] = He_surf
        
        print('Main He3 loop scenario ' + str(j) + 'complete.')
        
        if save_output == True:
            np.savetxt('D:/STEIN_paper/model_outputs/stochastic_erosion/STO_He3_ER' + str(ER) + '_surfmat.csv', 
                           He_surf,
                           delimiter = ',')
    
    return He_surf, He_surf_mat